Set up environment

In [ ]:
from copy import copy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
sns.set_style("white")

In [ ]:
import sys
sys.path.append('/Users/zeynepenkavi/Dropbox/RangelLab/NeuralMassModels/helpers')
from make_network import default_args, generateStructuralNetwork, generateSynapticNetwork, networkModel
from make_stimtimes import make_stimtimes
from get_pearson_fc_wci import get_pearson_fc_wci
from get_multreg_fc_wci import get_multreg_fc_wci
from run_glm import run_glm
from actflow import actflow

The goal of this network is to understand if/how activity flow mapping might work if the FC matrices are so different from underlying true connectivity

# Generate random networks

In [ ]:
num_sims = 20
networks = np.ndarray(num_sims, dtype=object)

In [ ]:
for i in range(num_sims):
    sim_args = copy(default_args)
    sim_args.update({'hubnetwork_dsity': .5,
                'outnetwork_dsity':.3, 
                'innetwork_dsity': .60,})
    W = generateSynapticNetwork(generateStructuralNetwork(sim_args), sim_args)
    sim_args.update({'W': W,
                'noise': 1, 
                'noise_scale': 1,
                'Tmax': 3000})
    networks[i] = sim_args

# Generate resting- and task-state time series

Resting data in this case is only noise not an AR(1) process.

In [ ]:
rest_data = np.ndarray(num_sims, dtype=object)

for i in range(num_sims):
    sim_args = networks[i]
    rest_data[i], _ = networkModel(sim_args['W'], args_dict=sim_args)

Task is a long infrequent stimulation task that stimulates only one node

In [ ]:
for i in range(num_sims):
    networks[i].update({'on_len': 64, 'off_len':32, 'stim_nodes':1, 'noise_scale': 0.2})
    
tasktiming, stimtimes, networks[0] = make_stimtimes(networks[0]['stim_nodes'], networks[0])

for i in range(num_sims):
    networks[i].update({'I': stimtimes,
                       'Tmax': len(tasktiming)})

task_data = np.ndarray(num_sims, dtype=object)

for i in range(num_sims):
    sim_args = networks[i]
    task_data[i], _ = networkModel(sim_args['W'], args_dict=sim_args)


Plot some networks' activity

In [ ]:
plt.rcParams["figure.figsize"][0] = 20
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 3)

for i,j in enumerate([3,5,8]):
    task1_data_df = pd.DataFrame(task_data[j])
    task1_data_df['node_num'] = task1_data_df.index
    task1_data_df = task1_data_df.melt(id_vars = ['node_num'])
    task1_data_df['node_num'] = task1_data_df["node_num"].astype('category')
    task1_data_df = task1_data_df.rename(columns={"variable": "time", "value": "activity"})
    task1_data_df = task1_data_df.query('time < 1000')
    sns.lineplot(data=task1_data_df, x="time", y="activity", hue="node_num", ax= a[i])

# Compute rest FC connectivity matrices

## Pearson

In [ ]:
rest_pearson_fcs = np.ndarray(num_sims, dtype=object)
rest_pearson_fcs_lb = np.ndarray(num_sims, dtype=object)
rest_pearson_fcs_ub = np.ndarray(num_sims, dtype=object)
rest_pearson_fcs_rmse = np.ndarray(num_sims, dtype=object)

for i, cur_data in enumerate(rest_data):
    rest_pearson_fcs[i], rest_pearson_fcs_lb[i], rest_pearson_fcs_ub[i], rest_pearson_fcs_rmse[i] = get_pearson_fc_wci(cur_data, networks[i])

## Multiple regression

In [ ]:
rest_mreg_fcs = np.ndarray(num_sims, dtype=object)
rest_mreg_fcs_lb = np.ndarray(num_sims, dtype=object)
rest_mreg_fcs_ub = np.ndarray(num_sims, dtype=object)
rest_mreg_fcs_rmse = np.ndarray(num_sims, dtype=object)

for i, cur_data in enumerate(rest_data):
    rest_mreg_fcs[i], rest_mreg_fcs_lb[i], rest_mreg_fcs_ub[i], rest_mreg_fcs_rmse[i] = get_multreg_fc_wci(cur_data, networks[i])

# Act flow mapping

<font color='red'>**Question: Does activity flow mapping consistently underestimate activation values (since the connectivity estimates are lower than true values)?**</font>

Actflow mapping has two ingredients
1. Task activations (GLM betas)
2. (Functional) connectivity matrix

Run task GLM on neural (not BOLD) level

In [ ]:
task_betas = np.ndarray(num_sims, dtype=object)

for i in range(num_sims):
    task_betas[i] = run_glm(task_data[i], tasktiming)


## Using resting FC

In [ ]:
actflow_preds = np.ndarray(num_sims, dtype=object)

for i in range(num_sims):
    actflow_preds[i] = actflow(task_betas[i], rest_pearson_fcs[i])

Plot predicted versus actual activations (scatterplot color by node)

In [ ]:
tmp = pd.DataFrame(np.vstack(task_betas))
tmp['network'] = list(range(num_sims))
tmp = pd.melt(tmp, id_vars = 'network', var_name = 'node', value_name = 'task_betas')

tmp2 = pd.DataFrame(np.vstack(actflow_preds))
tmp2['network'] = list(range(num_sims))
tmp2 = pd.melt(tmp2, id_vars = 'network', var_name = 'node', value_name = 'actflow_preds')

tmp = tmp.merge(tmp2, on=['network', 'node'])

plt.rcParams["figure.figsize"][0] = 7
plt.rcParams["figure.figsize"][1] = 5

sns.scatterplot(x='task_betas', y='actflow_preds', data = tmp, hue = 'node', palette='deep', s=70)
plt.axline([0, 0], [1, 1], color="black")
plt.xlabel("True task betas")
plt.ylabel("Actflow predicted betas")

Plot predicted versus actual activations as normalized heatmaps (a la Cole et al.)

In [ ]:
plt.rcParams["figure.figsize"][0] = 16
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 2)

sns.heatmap(pd.DataFrame(np.vstack(task_betas)), ax=a[0], vmin=-.05, vmax=1.2, cmap="vlag", annot=True)
a[0].set_title('True betas (raw)')
sns.heatmap(pd.DataFrame(np.vstack(actflow_preds)), ax=a[1], vmin=-.05, vmax=1.2, cmap="vlag", annot=True)
a[1].set_title('Actflow predictions (raw)')

Do things look better standardized (mean centered and unit variance)?

In [ ]:
plt.rcParams["figure.figsize"][0] = 16
plt.rcParams["figure.figsize"][1] = 5
fig, a = plt.subplots(1, 2)

sns.heatmap(pd.DataFrame(scale(np.vstack(task_betas))), ax=a[0], vmin=-2, vmax=3, cmap="vlag")
a[0].set_title('True betas (standardized)')
sns.heatmap(pd.DataFrame(scale(np.vstack(actflow_preds))), ax=a[1], vmin=-2, vmax=3, cmap="vlag")
a[1].set_title('Actflow predictions (standardized)')

<font color='red'>Is this what Cole et al. depict too? Or do they process the coefficients some other way?</font>

## Using task FC

How does task functional connectivity differ from rest FC?

In [ ]:
task_pearson_fcs = np.ndarray(num_sims, dtype=object)
task_pearson_fcs_lb = np.ndarray(num_sims, dtype=object)
task_pearson_fcs_ub = np.ndarray(num_sims, dtype=object)
task_pearson_fcs_rmse = np.ndarray(num_sims, dtype=object)
task_mreg_fcs = np.ndarray(num_sims, dtype=object)
task_mreg_fcs_lb = np.ndarray(num_sims, dtype=object)
task_mreg_fcs_ub = np.ndarray(num_sims, dtype=object)
task_mreg_fcs_rmse = np.ndarray(num_sims, dtype=object)


for i, cur_data in enumerate(task_data):
    task_pearson_fcs[i], task_pearson_fcs_lb[i], task_pearson_fcs_ub[i], task_pearson_fcs_rmse[i] = get_pearson_fc_wci(cur_data, networks[i])
    task_mreg_fcs[i], task_mreg_fcs_lb[i], task_mreg_fcs_ub[i], task_mreg_fcs_rmse[i] = get_multreg_fc_wci(cur_data, networks[i])

Does task FC do a better job in actflow?

## Sanity check: using true connectivity

does the true connectivity matrix do better in actflow?

## Using directed connectivity 

Can we improve on activity flow mapping with directed connectivity estimates from resting state?